In [1]:
# -- Import data from BigQuery -- #
import pandas_gbq

project_id = 'data-science-398321'
query = """
SELECT
  date,
  period,
  category,
  subcategory,
  job_index
FROM `data-science-398321.prod_marketing.jobs_online_monthly_unadjusted_series`
WHERE period IS NOT NULL # Exclude periods of high fluctuation
"""

df = pandas_gbq.read_gbq(query, project_id=project_id)


Downloading: 100%|██████████|


In [2]:
print(f"rows: {len(df)}")
df.head(5)


rows: 4681


,date,period,category,subcategory,job_index
0,2024-06-01,Downtrend (2023-NOW),Region,Auckland,58.6
1,2024-12-01,Downtrend (2023-NOW),Region,Auckland,36.7
2,2025-03-01,Downtrend (2023-NOW),Region,Auckland,62.7
3,2023-04-01,Downtrend (2023-NOW),Region,Auckland,92.8
4,2025-02-01,Downtrend (2023-NOW),Region,Auckland,57.0


In [3]:
# -- Seasonal decompose each column -- #
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import pandas as pd

def decompose_by_period_subcategory(
    df, period=12, model='additive',
    two_sided=True, extrapolate_trend=0):

    results = []

    # Group by period and category/subcategory
    for (p, cat, subcat), group in df.groupby(["period", "category", "subcategory"]):
        group = group.sort_values("date").set_index("date")

        result = seasonal_decompose(
            group["job_index"],
            model=model,
            period=period,
            two_sided=two_sided,
            extrapolate_trend=extrapolate_trend
        )

        # DataFrame
        tmp = pd.DataFrame({
            "date": group.index,
            "period": p,
            "category": cat,
            "subcategory": subcat,
            "observed": result.observed.values,
            "trend": result.trend.values,
            "seasonal": result.seasonal.values,
            "resid": result.resid.values
        })
        results.append(tmp)

    # Concat results
    results_df = pd.concat(results, ignore_index=True)
    results_df['date'] = pd.to_datetime(results_df['date']).dt.date
    return results_df


In [4]:
decompose_df = decompose_by_period_subcategory(df, extrapolate_trend=6)
decompose_df.head(5)

,date,period,category,subcategory,observed,trend,seasonal,resid
0,2023-01-01,Downtrend (2023-NOW),Industry,Business_services,83.2,86.681845,1.244684,-4.726529
1,2023-02-01,Downtrend (2023-NOW),Industry,Business_services,80.8,84.625000,1.136152,-4.961152
2,2023-03-01,Downtrend (2023-NOW),Industry,Business_services,95.3,82.568155,8.726827,4.005018
3,2023-04-01,Downtrend (2023-NOW),Industry,Business_services,74.1,80.511310,0.048057,-6.459367
4,2023-05-01,Downtrend (2023-NOW),Industry,Business_services,91.3,78.454464,10.026232,2.819304


In [10]:
# -- Push data to BigQuery -- #
from pandas_gbq import to_gbq

project_id = "data-science-398321"
dataset = "prod_marketing"
table = "jobs_online_monthly_seasonality"

to_gbq(
    decompose_df,
    destination_table=f"{dataset}.{table}",
    project_id=project_id,
    if_exists="replace"  # 或 "append"
)


100%|██████████| 1/1 [00:00<00:00, 3243.85it/s]


In [11]:
# Check the uploaded data
from google.cloud import bigquery

table_id = f"{project_id}.{dataset}.{table}"
sql = f"""
SELECT
  COUNT(*) AS n_rows,
FROM `{table_id}`
"""

check_df = pandas_gbq.read_gbq(query, project_id=project_id)


Downloading: 100%|██████████|


In [12]:
print(f"rows: {len(check_df)}")
check_df.head(5)


rows: 4681


,date,period,category,subcategory,job_index
0,2024-06-01,Downtrend (2023-NOW),Region,Auckland,58.6
1,2024-12-01,Downtrend (2023-NOW),Region,Auckland,36.7
2,2025-03-01,Downtrend (2023-NOW),Region,Auckland,62.7
3,2023-04-01,Downtrend (2023-NOW),Region,Auckland,92.8
4,2025-02-01,Downtrend (2023-NOW),Region,Auckland,57.0


In [13]:
# -- Check residual summary -- #
# If it's high, we may not see the seasonal patterns are stable
import pandas as pd
import numpy as np

def residual_summary(decompose_df, period, category):
    # specific category
    
    df = decompose_df[decompose_df["period"] == period].copy()
    df = df[df['category'] == category]
    
    # rmse
    summary = (df.groupby("subcategory")
               .apply(lambda g: pd.Series({
                   "rmse": np.sqrt(np.mean(g["resid"].dropna()**2)),
                   "mae": g["resid"].abs().mean(),
                   "mape": (g["resid"].abs() / g["observed"].replace(0, np.nan).abs()).mean() * 100,
                   "resid_to_obs": (
                       g.loc[g["resid"].notna(), "resid"].abs().sum() /
                       g.loc[g["resid"].notna(), "observed"].abs().sum()
                   ) * 100
               }))
               .reset_index())
    return summary


In [14]:
# Check the result
resid_check = residual_summary(decompose_df, period = 'Downtrend (2023-NOW)', category="Industry")
print(resid_check.sort_values("rmse", ascending=False).head(10))


         subcategory       rmse        mae      mape  resid_to_obs
3        Health_care  32.872051  24.689469  9.209903      8.782961
7              Other  18.116702  16.095008  6.352322      5.884552
8            Primary  17.776673  13.887008  4.214494      4.406182
2          Education  16.063218  10.289893  4.252693      4.765049
4        Hospitality  14.377056  11.251827  5.812999      5.524250
1       Construction  13.147847  11.001809  5.974290      5.621680
6      Manufacturing  11.800480   8.697742  4.532985      4.162691
9              Sales   5.270070   4.342585  5.060340      4.704530
0  Business_services   3.848629   3.254787  5.811908      5.381821
5                 IT   2.609767   2.087974  5.822855      5.416047


/var/tmp/ipykernel_7637/2015711376.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({
